# Pending Task: Fix VQQNN model

In [ ]:
#from https://developers.refinitiv.com/en/article-catalog/article/tensorflow-variational-quantum-neural-networks-in-finance
#In this cell data is obtained from yfinance
import os
import yfinance as yf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import QuantumDense
import datetime as date
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.preprocessing import MinMaxScaler
import pytz


APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, APCA_API_NEWS_URL = "", "", "", ""

pd.set_option('display.max_columns', None)

ticker = input("Enter stocks to analyze: ")



    
now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')




def set_vars(secrets_file="secrets-alpaca.env"):
    """
    This function initializes the credentials by setting APCA_API_KEY_ID, APCA_API_SECRET_KEY, and APCA_API_BASE_URL
    global variables from a specified .env file in the same directory

    Parameters
    ----------
    secrets_file: Default set to secrets-alpaca.env, can be any secrets file containing the

    Returns
    -------
    None

    """

    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL

    with open(f"{secrets_file}", 'r') as file:
        contents = file.read()
        env_vars = contents.replace('export ', '').split("\n")
        APCA_API_KEY_ID = env_vars[0].split("=")[1]
        APCA_API_SECRET_KEY = env_vars[1].split("=")[1]
        APCA_API_BASE_URL = env_vars[2].split("=")[1]
        APCA_API_NEWS_URL = env_vars[2].split("=")[1]


def primaryfunc():
    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL
    set_vars()
    api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL)
    #test1 = api.get_aggs("INTC", 1, 1,'2023-03-21','2023-03-31')
    df = api.get_bars(ticker, TimeFrame.Hour, "2020-01-26", "2023-04-10", adjustment='raw').df



    df['RETURNS'] = df['close'].pct_change()
    df['PRX_MA_ND'] = df['close'].rolling(window=5).mean()
    df['VOLATILITY'] = df['close'].rolling(window=5).std()
    df['TP1_RETURNS'] = df['RETURNS'].shift(-1)
    df['VOLUME'] = df['volume']



    if df.isnull().sum().sum() > 0:
        df.dropna(inplace=True)
    else:
        print("No rows with complete data found in the dataframe")

    df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
    df_y = df['TP1_RETURNS']

    df_x_scaler = MinMaxScaler().fit(df_x)

    forward_test_date = pd.to_datetime('2023-04-11').tz_localize(pytz.utc)
    x_train = np.array([], dtype=np.float32)
    y_train = np.array([], dtype=np.float32)
    x_test = np.array([], dtype=np.float32)
    y_test = np.array([], dtype=np.float32)
    if forward_test_date > df_x.index.max().tz_convert(pytz.utc):
        print("Test date is outside the range of the dataframe")
    else:
        fdf_x = df_x.loc[forward_test_date:]
        fdf_y = df_y.loc[forward_test_date:]
        df_x = df_x.loc[:forward_test_date]
        df_y = df_y.loc[:forward_test_date]

        df_x_scaled = pd.DataFrame(df_x_scaler.transform(df_x))
        fdf_x_scaled = pd.DataFrame(df_x_scaler.transform(fdf_x))
        
 

        x_train, x_test, y_train, y_test = train_test_split(df_x_scaled,
                                                            df_y,
                                                            test_size=0.25,
                                                            random_state=42)
    if forward_test_date <= df_x.index.max().tz_convert(pytz.utc):
        x_train, x_test, y_train, y_test = train_test_split(df_x_scaled,
                                                        df_y,
                                                        test_size=0.25,
                                                        random_state=42)
        x_train = np.expand_dims(x_train.values, 1).astype(np.float32)
        y_train = np.expand_dims(y_train.values, 1).astype(np.float32)
        x_validation = np.expand_dims(x_test.values, 1).astype(np.float32)
        y_validation = np.expand_dims(y_test.values, 1).astype(np.float32)
    else:
        print("Test date is outside the range of the dataframe")


    x_train = np.expand_dims(x_train, 1).astype(np.float32)
    y_train = np.expand_dims(y_train, 1).astype(np.float32)
    x_validation = np.expand_dims(x_test, 1).astype(np.float32)
    y_validation = np.expand_dims(y_test, 1).astype(np.float32)

    qnn_model = VQNNModel()
    qnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                         beta_1=0.9,
                                                         beta_2=0.999,
                                                         epsilon=1e-07),
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=["mean_squared_error"],
                      run_eagerly=True)

    log_dir = "logs\\model\\eikon\\"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    qnn_model.fit(
    x_train, y_train, epochs=1, batch_size=1,
                  callbacks=[tensorboard_callback])


    num_of_features = fdf_x_scaled.shape[1]
    qnn_predictions = []
    for entry in fdf_x_scaled.iterrows():
        fdf_x_predict_tensor = tf.reshape(tf.convert_to_tensor(entry[1]), [1, num_of_features])
        qnn_forecast = qnn_model.predict(fdf_x_predict_tensor)
        qnn_predictions.append(qnn_forecast[-1, -1, -1])
    signal = [0 if x <= 0 else 1 for x in qnn_predictions]
    
    


    sub_qr = QuantumRegister(10)
    sub_circuit = QuantumCircuit(sub_qr, name='sub_circ')
    sub_circuit.crz(1, sub_qr[0], sub_qr[5])
    sub_circuit.barrier()

    # Convert to a gate instruction and connect it with the QuantumDense registers
    sub_instructions = sub_circuit.to_instruction()

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        QuantumDense(qubits=10, instructions=sub_instructions)
    ])


    print(df)


primaryfunc()




In [ ]:
pip install pytz

In [2]:
#This cell is meant to debug cell above
#from https://developers.refinitiv.com/en/article-catalog/article/tensorflow-variational-quantum-neural-networks-in-finance
import os
import yfinance as yf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from tensor_layer_expermental import QuantumDense
import datetime as date
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, APCA_API_NEWS_URL = "", "", "", ""

pd.set_option('display.max_columns', None)

ticker = "INTC"



    
now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

def set_vars(secrets_file="secrets-alpaca.env"):
    """
    This function initializes the credentials by setting APCA_API_KEY_ID, APCA_API_SECRET_KEY, and APCA_API_BASE_URL
    global variables from a specified .env file in the same directory

    Parameters
    ----------
    secrets_file: Default set to secrets-alpaca.env, can be any secrets file containing the

    Returns
    -------
    None

    """

    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL

    with open(f"{secrets_file}", 'r') as file:
        contents = file.read()
        env_vars = contents.replace('export ', '').split("\n")
        APCA_API_KEY_ID = env_vars[0].split("=")[1]
        APCA_API_SECRET_KEY = env_vars[1].split("=")[1]
        APCA_API_BASE_URL = env_vars[2].split("=")[1]
        APCA_API_NEWS_URL = env_vars[2].split("=")[1]


def primaryfunc():
    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL
    set_vars()
    api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL)
    #test1 = api.get_aggs("INTC", 1, 1,'2023-03-21','2023-03-31')
    df = api.get_bars(ticker, TimeFrame.Hour, "2023-03-31", "2023-03-31", adjustment='raw').df
    
    
    df['HL_DELTA'] = df['high'] - df['low']
    df['RETURNS'] = df['close'].pct_change()
    df['PRX_MA_ND'] = df['close'].rolling(window=5).mean()
    df['VOLATILITY'] = df['close'].rolling(window=5).std()
    df['TP1_RETURNS'] = df['RETURNS'].shift(-1)

    df.dropna(inplace=True)
    print(df)
    
primaryfunc()




                              open     high     low   close    volume  \
timestamp                                                               
2023-03-31 12:00:00+00:00  32.0700  32.2200  31.910  31.980    193101   
2023-03-31 13:00:00+00:00  31.9800  32.3300  31.730  32.110   5734329   
2023-03-31 14:00:00+00:00  32.1100  32.6200  32.110  32.585  10117957   
2023-03-31 15:00:00+00:00  32.5800  32.6800  32.520  32.590   5716578   
2023-03-31 16:00:00+00:00  32.5850  32.9000  32.500  32.820   5537337   
2023-03-31 17:00:00+00:00  32.8150  32.8700  32.705  32.775   4641895   
2023-03-31 18:00:00+00:00  32.7739  32.8650  32.615  32.675   5024295   
2023-03-31 19:00:00+00:00  32.6750  32.8000  32.530  32.645  11803141   
2023-03-31 20:00:00+00:00  32.6700  32.6996  32.560  32.640   1655629   
2023-03-31 21:00:00+00:00  32.6400  32.6500  32.620  32.640      6960   
2023-03-31 22:00:00+00:00  32.6200  32.6800  32.610  32.680      9663   

                           trade_count       vwap 

In [ ]:
#from https://developers.refinitiv.com/en/article-catalog/article/tensorflow-variational-quantum-neural-networks-in-finance
#In this cell data is obtained from yfinance
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import QuantumDense
import datetime as date
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.preprocessing import MinMaxScaler
import pytz
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation

class VQNNModel(tf.keras.Model):
    def __init__(self, num_of_features):
        super(VQNNModel, self).__init__()
        self.num_of_features = num_of_features
        self.input_layer = Input(shape=(num_of_features,))
        self.hidden_layer_1 = Dense(128)(self.input_layer)
        self.activ_1 = Activation('relu')(self.hidden_layer_1)
        self.hidden_layer_2 = Dense(64)(self.activ_1)
        self.activ_2 = Activation('tanh')(self.hidden_layer_2)
        self.hidden_layer_3 = Dense(32)(self.activ_2)
        self.activ_3 = Activation('elu')(self.hidden_layer_3)
        self.output_layer = Dense(1)(self.activ_3)

    def call(self, inputs):
        x = self.input_layer(inputs)
        x = self.hidden_layer_1(x)
        x = self.activ_1(x)
        x = self.hidden_layer_2(x)
        x = self.activ_2(x)
        x = self.hidden_layer_3(x)
        x = self.activ_3(x)
        x = self.output_layer(x)
        return x



pd.set_option('display.max_columns', None)


    
now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')


def primaryfunc():
    pd.set_option('display.max_columns', None)

    now = date.date.today()
    today = now.strftime('%Y-%m-%d')
    yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
    ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

    # read input data from CSV file
    df = pd.read_csv('datasets/EHC_5yr.csv')

    # convert date column to datetime and set as index
    df['DATE'] = pd.to_datetime(df['DATE'])
    df.set_index('DATE', inplace=True)

    # localize the timestamps to UTC, if not already timezone-aware
    df.index = df.index.tz_localize(pytz.utc)

    df['RETURNS'] = df['CLOSE'].pct_change()
    df['PRX_MA_ND'] = df['CLOSE'].rolling(window=5).mean()
    df['VOLATILITY'] = df['CLOSE'].rolling(window=5).std()
    df['TP1_RETURNS'] = df['RETURNS'].shift(-1)
    df['VOLUME'] = df['VOLUME']

    df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
    df_y = df['TP1_RETURNS']

    df_x_scaler = MinMaxScaler().fit(df_x)

    forward_test_date = pd.to_datetime('2023-04-10').tz_localize(pytz.utc)

    df_x_scaled = pd.DataFrame()
    fdf_x_scaled = pd.DataFrame()
    if forward_test_date > df_x.index.max():
        print("Test date is outside the range of the dataframe")
        fdf_x = pd.DataFrame()
        fdf_y = pd.Series()
    else:
        fdf_x = df_x.loc[forward_test_date:]
        fdf_y = df_y.loc[forward_test_date:]
        df_x = df_x.loc[:forward_test_date]
        df_y = df_y.loc[:forward_test_date]

        df_x_scaled = pd.DataFrame(df_x_scaler.transform(df_x))
        fdf_x_scaled = pd.DataFrame(df_x_scaler.transform(fdf_x))

    if not df_x_scaled.empty and len(df_x_scaled) == len(df_y):
        x_train, x_test, y_train, y_test = train_test_split(df_x_scaled, df_y, test_size=0.25, random_state=42)

        if x_train is None:
            print("x_train is None, returning default value")
            num_of_features = df_x.shape[1] # <-- Define num_of_features here
            x_train = np.zeros((1, num_of_features, 1)).astype(np.float32)
            y_train = np.zeros((1, 1, 1)).astype(np.float32)
        else:
            num_of_features = df_x.shape[1] # <-- Define num_of_features here
            x_train = np.expand_dims(x_train.values, 1).astype(np.float32)
            y_train = np.expand_dims(y_train.values, 1).astype(np.float32)
            x_validation = np.expand_dims(x_test.values, 1).astype(np.float32)
            y_validation = np.expand_dims(y_test.values, 1).astype(np.float32)

        qnn_model = VQNNModel(num_of_features)
        qnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                             beta_1=0.9,
                                                             beta_2=0.999,
                                                             epsilon=1e-07),
                          loss=tf.keras.losses.MeanSquaredError(),
                          metrics=["mean_squared_error"])

        # Fit the model to the training data
        qnn_model.fit(
            x_train, y_train, epochs=1, batch_size=1
        )

        # Define the number of features for the forward test data
        num_of_features = fdf_x_scaled.shape[1]

        # Use the trained model to make predictions
        num_of_features = fdf_x_scaled.shape[1]

        qnn_predictions = []
        for entry in fdf_x_scaled.iterrows():
            fdf_x_predict_tensor = tf.reshape(tf.convert_to_tensor(entry[1]), [1, num_of_features])
            qnn_forecast = qnn_model.predict(fdf_x_predict_tensor)
            qnn_predictions.append(qnn_forecast[-1, -1, -1])

        signal = [0 if x <= 0 else 1 for x in qnn_predictions]

        sub_qr = QuantumRegister(10)
        sub_circuit = QuantumCircuit(sub_qr, name='sub_circ')
        sub_circuit.crz(1, sub_qr[0], sub_qr[5])
        sub_circuit.barrier()

        # Convert to a gate instruction and connect it with the QuantumDense registers
        sub_instructions = sub_circuit.to_instruction()

        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            QuantumDense(qubits=10, instructions=sub_instructions)
        ])

        log_dir = "logs\\model\\eikon\\"
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


primaryfunc()




In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import QuantumDense
import datetime as date
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.preprocessing import MinMaxScaler
import pytz
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation


class VQNNModel(tf.keras.Model):
    def __init__(self, num_of_features):
        super(VQNNModel, self).__init__()
        self.num_of_features = num_of_features
        self.input_layer = Input(shape=(num_of_features,))
        self.hidden_layer_1 = Dense(128)(self.input_layer)
        self.activ_1 = Activation('relu')(self.hidden_layer_1)
        self.hidden_layer_2 = Dense(64)(self.activ_1)
        self.activ_2 = Activation('tanh')(self.hidden_layer_2)
        self.hidden_layer_3 = Dense(32)(self.activ_2)
        self.activ_3 = Activation('elu')(self.hidden_layer_3)
        self.output_layer = Dense(1)(self.activ_3)

    def call(self, inputs):
        x = self.input_layer(inputs)
        x = self.hidden_layer_1(x)
        x = self.activ_1(x)
        x = self.hidden_layer_2(x)
        x = self.activ_2(x)
        x = self.hidden_layer_3(x)
        x = self.activ_3(x)
        x = self.output_layer(x)
        return x

def primaryfunc():
    pd.set_option('display.max_columns', None)

    now = date.date.today()
    today = now.strftime('%Y-%m-%d')
    yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
    ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

    # read input data from CSV file
    df = pd.read_csv('datasets/EHC_5yr.csv')

    # convert date column to datetime and set as index
    df['DATE'] = pd.to_datetime(df['DATE'])
    df.set_index('DATE', inplace=True)
    df['HL_DELTA'] = df['HIGH'] - df['LOW']
    df['RETURNS'] = df['CLOSE'].pct_change()
    df['PRX_MA_ND'] = df['CLOSE'].rolling(window=5).mean()
    df['VOLATILITY'] = df['CLOSE'].rolling(window=5).std()
    df['TP1_RETURNS'] = df['RETURNS'].shift(-1)

    df.dropna(inplace=True)
    
    # localize the timestamps to UTC, if not already timezone-aware
    df.index = df.index.tz_localize(pytz.utc)

    df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
    df_y = df['TP1_RETURNS']

    df_x_scaler = MinMaxScaler().fit(df_x)

    forward_test_date = pd.to_datetime('2025-03-01').tz_localize(pytz.utc)

    df_x_scaled = pd.DataFrame()
    fdf_x_scaled = pd.DataFrame()
    if forward_test_date > df_x.index.max():
        print("Test date is outside the range of the dataframe")
        fdf_x = pd.DataFrame()
        fdf_y = pd.Series()
    else:
        fdf_x = df_x.loc[forward_test_date:]
        fdf_y = df_y.loc[forward_test_date:]
        df_x = df_x.loc[:forward_test_date]
        df_y = df_y.loc[:forward_test_date]

        df_x_scaled = pd.DataFrame(df_x_scaler.transform(df_x))
        fdf_x_scaled = pd.DataFrame(df_x_scaler.transform(fdf_x))

    if not df_x_scaled.empty and len(df_x_scaled) == len(df_y):
        x_train, x_test, y_train, y_test = train_test_split(df_x_scaled, df_y, test_size=0.25, random_state=42)

        if x_train is None:
            print("x_train is None, returning default value")
            num_of_features = df_x.shape[1] # <-- Define num_of_features here
            x_train = np.zeros((1, num_of_features, 1)).astype(np.float32)
            y_train = np.zeros((1, 1, 1)).astype(np.float32)
        else:
            num_of_features = df_x.shape[1] # <-- Define num_of_features here
            x_train = np.expand_dims(x_train.values, 1).astype(np.float32)
            y_train = np.expand_dims(y_train.values, 1).astype(np.float32)
            x_validation = np.expand_dims(x_test.values, 1).astype(np.float32)
            y_validation = np.expand_dims(y_test.values, 1).astype(np.float32)

        qnn_model = VQNNModel(num_of_features)
        qnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                             beta_1=0.9,
                                                             beta_2=0.999,
                                                             epsilon=1e-07),
                          loss=tf.keras.losses.MeanSquaredError(),
                          metrics=["mean_squared_error"])

        # Fit the model to the training data
        qnn_model.fit(
            x_train, y_train, epochs=1, batch_size=1
        )

                # Define the number of features for the forward test data
        num_of_features = fdf_x_scaled.shape[1]

        qnn_predictions = []
        for entry in fdf_x_scaled.iterrows():
            fdf_x_predict_tensor = tf.reshape(tf.convert_to_tensor(entry[1]), [1, num_of_features])
            qnn_forecast = qnn_model.predict(fdf_x_predict_tensor)
            qnn_predictions.append(qnn_forecast[-1, -1, -1])

        signal = [0 if x <= 0 else 1 for x in qnn_predictions]
        sub_qr = QuantumRegister(10)
        sub_circuit = QuantumCircuit(sub_qr, name='sub_circ')
        sub_circuit.crz(1, sub_qr[0], sub_qr[5])
        sub_circuit.barrier()

        # Convert to a gate instruction and connect it with the QuantumDense registers
        sub_instructions = sub_circuit.to_instruction()

        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            QuantumDense(qubits=10, instructions=sub_instructions)
        ])

        log_dir = "logs\\model\\eikon\\"
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

primaryfunc()

In [4]:
#Master Code
#Source and credit: https://github.com/MSKX/tensorflow-quantum_dense
#Confirm successful import of packages
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as plt_dates

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from QuantumDense import VQNNModel

import matplotlib.dates as plt_dates

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from QuantumDense import VQNNModel

#Confirm successful import of input data, instead of refinitive.dataplatform.eikon API
import yfinance as yf
import pandas as pd
import datetime as date


now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

# read input data from CSV file
df = pd.read_csv('datasets/test.csv')

# convert date column to datetime and set as index
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
df['HL_DELTA'] = df['HIGH'] - df['LOW']
df['RETURNS'] = df['CLOSE'].pct_change()
df['PRX_MA_ND'] = df['CLOSE'].rolling(window=5).mean()
df['VOLATILITY'] = df['CLOSE'].rolling(window=5).std()
df['TP1_RETURNS'] = df['RETURNS'].shift(-1)

df.dropna(inplace=True)

#Create 3 dimensional Tensor
df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
df_y = df['TP1_RETURNS']
df_x.dropna(inplace=True)

df_x_scaler = MinMaxScaler().fit(df_x)

forward_test_date = '2022-10-01'

fdf_x = df_x.loc[forward_test_date:]
fdf_y = df_y.loc[forward_test_date:]
df_x = df_x.loc[:forward_test_date]
df_y = df_y.loc[:forward_test_date]

fdf_prx = df.loc[forward_test_date:]['CLOSE']
fdf_y_len = len(fdf_y)

df_x_scaled = pd.DataFrame(df_x_scaler.transform(df_x))
fdf_x_scaled = pd.DataFrame(df_x_scaler.transform(fdf_x))

x_train, x_test, y_train, y_test = train_test_split(df_x_scaled,
                                                    df_y,
                                                    test_size=0.25,
                                                    random_state=42)

x_train = np.expand_dims(x_train.values, 1).astype(np.float32)
y_train = np.expand_dims(y_train.values, 1).astype(np.float32)
x_validation = np.expand_dims(x_test.values, 1).astype(np.float32)
y_validation = np.expand_dims(y_test.values, 1).astype(np.float32)

qnn_model = VQNNModel()
qnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                     beta_1=0.9,
                                                     beta_2=0.999,
                                                     epsilon=1e-07),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=["mean_squared_error"])

qnn_model.run_eagerly = True

##patch
# assuming `df_x_test` is your test data
# create a scaler object and fit it on `x_train` data
scaler = MinMaxScaler().fit(x_train)

# use `transform()` to scale `df_x_test` using the same scaling parameters as `x_train`
x_test_scaled = scaler.transform(df_x_test)

# assuming `qnn_model` is your trained model
# use `predict()` to make predictions on the scaled test data
y_pred = qnn_model.predict(x_test_scaled)







log_dir = "logs\\model\\eikon\\"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

qnn_model.fit(
x_train, y_train, epochs=1, batch_size=1,
              callbacks=[tensorboard_callback])



C:\Users\Spark\AppData\Local\Temp\ipykernel_25812\2454655176.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x.dropna(inplace=True)


ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# assuming `df_x_test` is your test data
# create a scaler object and fit it on `x_train` data
scaler = MinMaxScaler().fit(x_train)

# use `transform()` to scale `df_x_test` using the same scaling parameters as `x_train`
x_test_scaled = scaler.transform(df_x_test)

# assuming `qnn_model` is your trained model
# use `predict()` to make predictions on the scaled test data
y_pred = qnn_model.predict(x_test_scaled)